# Downloading Files from GenBank

## Trying...

In [3]:
from Bio import Entrez
from Bio import SeqIO

Entrez.email = "fabien.allemand@etu.unistra.fr"

orga = "Homo sapiens"
cible = "gene"
db = "nuccore"
db = "nucleotide"
handle = Entrez.esearch(db=db, term="(" + orga + "[Organism] AND NC_*[Accession])", retmax ="99999999", usehistory='y')
record = Entrez.read(handle)
handle.close()
ids = record["IdList"]
print(len(ids))
print(ids)
for id in ids:
    handle = Entrez.efetch(db=db, id=id, rettype="gb", retmode="text")
    record = SeqIO.read(handle, "genbank")
    handle.close()
    print(record)
    Dna_seq = record.seq
    print(Dna_seq)
    cible_positions = []
    for feature in record.features:
        print(feature.type)
        if feature.type == cible:
            cible_positions.append((feature.location.start, feature.location.end))
            print(feature.location.start, feature.location.end)



24
['568815597', '568815596', '568815595', '568815594', '568815593', '568815592', '568815591', '568815590', '568815589', '568815588', '568815587', '568815586', '568815585', '568815584', '568815583', '568815582', '568815581', '568815580', '568815579', '568815578', '568815577', '568815576', '568815575', '568815574']
ID: NC_000001.11
Name: NC_000001
Description: Homo sapiens chromosome 1, GRCh38.p14 Primary Assembly
Database cross-references: BioProject:PRJNA168, Assembly:GCF_000001405.40
Number of features: 1
/molecule_type=DNA
/topology=linear
/data_file_division=CON
/date=20-MAR-2023
/accessions=['NC_000001']
/sequence_version=11
/keywords=['RefSeq']
/source=Homo sapiens (human)
/organism=Homo sapiens
/taxonomy=['Eukaryota', 'Metazoa', 'Chordata', 'Craniata', 'Vertebrata', 'Euteleostomi', 'Mammalia', 'Eutheria', 'Euarchontoglires', 'Primates', 'Haplorrhini', 'Catarrhini', 'Hominidae', 'Homo']
/references=[Reference(title='The DNA sequence and biological annotation of human chromosome 1

UndefinedSequenceError: Sequence content is undefined